In [29]:
import pandas as pd
import numpy as np
from random import randint

In [63]:
main_df = pd.read_csv(r'csv/main.csv')
def_att_df = pd.read_csv(r'csv/default_attributes.csv')

# remove rows without an item number
main_df = main_df[main_df['item_number'].notna()]#.head(1)

# example command:
# /give @p iron_axe{
    # Unbreakable:1b,
    # mole_item:1b,
    # Enchantments:[
        # {id:"minecraft:sharpness",lvl:4s},
        # {id:"minecraft:knockback",lvl:1s},
        # {id:"minecraft:fire_aspect",lvl:0s}
        # ],
    # AttributeModifiers:[
        # {AttributeName:"generic.movement_speed",
        #   Name:"generic.movement_speed",
        #   Amount:0.1,
        #   Operation:1,
        #   UUID:[I;466261690,875251594,-2069553399,107400599],
        #   Slot:"mainhand"}
        # ]
        # } 1

# convert enchants into json objects
def enchant_json(enchantment, level):
    if type(enchantment) == str:
        enchant_string = '{id:"minecraft:' + str(enchantment) + '",lvl:' + str(int(level) - 1) + 's}'
        return enchant_string
    else:
        return enchantment

main_df['enchant_json'] = main_df.apply(lambda x : enchant_json(x.enchant, x.level), axis=1)

# convert attributes into json objects and make random UUIDs, use default attributes of item if specified
def att_json(use_default, item_id, attributes, value, slot, operation):
    op_dict = {'amount':0, 'percentage':1, 'multiplicative':2}
    att_list = []
    try:
        op_type = op_dict[operation]
    except:
        op_type = 1
    if use_default == True:
        default_atts = def_att_df[def_att_df['item_id'] == item_id].drop('item_id', axis=1)
        att_slot = default_atts['slot'].values
        default_atts = default_atts.drop('slot', axis=1)
        for item in default_atts.columns:
            att = default_atts[item]
            att_val = str(float(att.values))
            if att_val!='nan':
                att_name = att.name
                # adjust attack speed/damage attributes to adjust for default speeds, this is only needed if there are other attributes
                if att_name == 'attack_speed' & str(attributes) != 'nan':
                    att_val = float(att.values) - 4
                if att_name == 'attack_damage' & str(attributes) != 'nan':
                    att_val = float(att.values) - 1
                att_val = str(att_val)
                UUID_val_1 = str(randint(100000000,999999999))
                str_out = '{AttributeName:"generic.' + att_name + '",Name:"generic.' + att_name + '",Amount:' + att_val + ',Operation:0,UUID:['+ UUID_val_1 +',875251594,-2069553399,107400599],Slot:"' + att_slot + '"}'
                att_list.append(str(str_out[0]))
    
    if str(attributes)!='nan':
        UUID_val_2 = str(randint(100000000,999999999))
        str_out = '{AttributeName:"generic.' + attributes + '",Name:"generic.' + attributes + '",Amount:' + str(value) + ',Operation:'+ str(op_type) +',UUID:[875251594,'+ UUID_val_2 +',-2069553399,107400599],Slot:"' + slot + '"}'
        att_list.append(str_out)
    
    return ','.join(att_list)
                

main_df['attribute_json'] = main_df.apply(lambda x : att_json(x.use_default_attributes,
                                                              x.item_type,
                                                              x.attributes,
                                                              x.value,
                                                              x.slot,
                                                              x.operation), 
                                          axis=1)


# group together rows which have the same item number, convert json objects (eg. enchants) into lists
# main_df.groupby(10)
# main_df.groupby('item_number').first().reset_index()

json_df = main_df[['item_number', 'enchant_json', 'attribute_json']].groupby('item_number').agg(list).reset_index()
main_df = main_df['item_number', 'item_type', 'item_name', 'item_lore', ].groupby('item_number').first().reset_index()
main_df = pd.merge(main_df, json_df, on='item_number', how='inner')
main_df.head()



,item_number,item_type,item_name,item_tier,item_lore,use_default_attributes,unbreakable,attributes,slot,operation,value,enchant,level,hide_enchant,custom_ability,enchant_json_x,attribute_json_x,enchant_json_y,attribute_json_y
0,1.0,iron_axe,Moleman's Hatchet,4.0,Wreaking havok with a hatchet from the molehole,True,False,None,None,None,NaN,fire_aspect,1.0,False,helmet remover,"{id:""minecraft:fire_aspect"",lvl:0s}","{AttributeName:""generic.attack_damage"",Name:""g...","[{id:""minecraft:fire_aspect"",lvl:0s}, {id:""min...","[{AttributeName:""generic.attack_damage"",Name:""..."
1,2.0,iron_hoe,Render's Mom,1.0,"Good joke man, real original",False,True,attack_speed,offhand,amount,3.0,aqua_affinity,1.0,True,None,"{id:""minecraft:aqua_affinity"",lvl:0s}","{AttributeName:""generic.attack_speed"",Name:""ge...","[{id:""minecraft:aqua_affinity"",lvl:0s}]","[{AttributeName:""generic.attack_speed"",Name:""g..."


In [42]:
test_df = def_att_df[def_att_df['item_id'] == 'iron_axe'].drop('item_id', axis=1)
test_df.columns

Index(['attack_damage', 'attack_speed', 'armor', 'armor_toughness',
       'knockback_resistance', 'slot'],
      dtype='object')